In [2]:
from principal_DBN_alpha import *
from principal_RBM_alpha import *
from utils import *
from tqdm import tqdm, notebook
import matplotlib.pyplot as plt

# Étude sur MNIST

## Chargement des données MNIST et processing

In [ ]:
from principal_DNN_MNIST import *

In [2]:
import torchvision.datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


# if not exist, download mnist dataset
train_set = torchvision.datasets.MNIST(root='./data', train=True, transform=None, download=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, transform=None, download=True)
batch_size = 100

train_set.data = (train_set.data > 127).float()
test_set.data = (test_set.data > 127).float()


# Applatir les images (28*28) en vecteurs (784)
train_mnist = train_set.data.view(train_set.data.shape[0], -1).double().to(device)
test_mnist = test_set.data.view(test_set.data.shape[0], -1).double().to(device) 


In [ ]:
# One Hot Encoding des labels
labels_train_mnist = torch.nn.functional.one_hot(train_set.targets).float().to(device)
labels_test_mnist = torch.nn.functional.one_hot(test_set.targets).float().to(device)


train_mnist.shape, labels_train_mnist.shape, test_mnist.shape, labels_test_mnist.shape


In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [20]:
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)





## Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(512, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        return x

# initialize the NN
model = Net()



# define the same model with Sequential
model = nn.Sequential(nn.Linear(28 * 28, 512),
                      nn.Sigmoid(),
                        nn.Linear(512, 512),
                        nn.Sigmoid(),
                        nn.Linear(512, 10),
                        nn.Softmax(dim=1),
                        nn.Dropout(0.2),)
print(model)

## Specify loss and optimization functions

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)



# number of epochs to train the model
n_epochs = 30  # suggest training between 20-50 epochs

model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data.view(data.shape[0], -1))
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))



Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=512, out_features=10, bias=True)
  (5): Softmax(dim=1)
  (6): Dropout(p=0.2, inplace=False)
)
Epoch: 1 	Training Loss: 2.303018


KeyboardInterrupt: 

In [13]:
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

for data, target in train_loader:
    print(data.view(-1, 28 * 28).shape)
    print(target.shape)
    break

torch.Size([300, 784])
torch.Size([300])


In [14]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
batch_size = 300

train_set.data = (train_set.data > 127).float()
test_set.data = (test_set.data > 127).float()


# Applatir les images (28*28) en vecteurs (784)
train_mnist = train_set.data.view(train_set.data.shape[0], -1).float().to(device)
test_mnist = test_set.data.view(test_set.data.shape[0], -1).float().to(device) 
labels_train_mnist = torch.nn.functional.one_hot(train_set.targets).float().to(device)
labels_test_mnist = torch.nn.functional.one_hot(test_set.targets).float().to(device)

dataset = torch.utils.data.TensorDataset(train_mnist, labels_train_mnist)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for data, target in loader:
    print(data.shape)
    print(target.shape)
    break

torch.Size([300, 784])
torch.Size([300, 10])


In [10]:


# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))



Test Loss: 0.273756

Test Accuracy of     0: 98% (962/980)
Test Accuracy of     1: 97% (1109/1135)
Test Accuracy of     2: 89% (919/1032)
Test Accuracy of     3: 90% (913/1010)
Test Accuracy of     4: 92% (912/982)
Test Accuracy of     5: 87% (780/892)
Test Accuracy of     6: 94% (910/958)
Test Accuracy of     7: 92% (947/1028)
Test Accuracy of     8: 88% (862/974)
Test Accuracy of     9: 90% (915/1009)

Test Accuracy (Overall): 92% (9229/10000)


In [9]:
epochs = 100
batch_size = 300
import torchvision.datasets
import torchvision.transforms as transforms
transform = transforms.ToTensor()

train_set = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
batch_size = 300

train_set.data = (train_set.data > 127).float()
test_set.data = (test_set.data > 127).float()


# Applatir les images (28*28) en vecteurs (784)
train_mnist = train_set.data.view(train_set.data.shape[0], -1).float().to(device)
test_mnist = test_set.data.view(test_set.data.shape[0], -1).float().to(device) 
labels_train_mnist = torch.nn.functional.one_hot(train_set.targets).float().to(device)
labels_test_mnist = torch.nn.functional.one_hot(test_set.targets).float().to(device)

dataset = torch.utils.data.TensorDataset(train_mnist, labels_train_mnist)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [11]:
def build_model_2():
        model = nn.Sequential(
            nn.Linear(784, 200),
            nn.Sigmoid(),
            nn.Linear(200, 10),
            nn.Softmax(dim=1)  
        )
        return model.to(device)
    
model = build_model_2()
model.train()
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

X = train_mnist

Y = labels_train_mnist

if batch_size is None:
    batch_size = int(np.floor(0.2 * X.shape[0]))
    
dataset = torch.utils.data.TensorDataset(X, Y)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
loss_list = []
epochs_iterator = tqdm(range(epochs), desc="Training DNN", unit="epoch")
for e in epochs_iterator:
    total_loss = 0.0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_hat = model(X_batch)
        loss = criterion(y_hat, y_batch)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        total_loss += loss.item()
    epochs_iterator.set_postfix({"Loss": loss.item()})


Training DNN:  10%|█         | 10/100 [00:09<01:26,  1.04epoch/s, Loss=2.29]


KeyboardInterrupt: 

## Test du DNN sur MNIST

In [ ]:
nb_pixels = train_mnist.shape[1]
neurons = 200
epochs_pretrain = 100
epochs_backprop = 100
learning_rate = 0.2
nb_layers = 2
layers_dbn = [nb_pixels] + [neurons]*(nb_layers-1)
nb_classes = len(train_set.class_to_idx)
batch_size = 0.005*len(train_mnist)
batch_size = int(batch_size)

### DNN pre-entrainement

In [ ]:
dnn_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
dnn_pretrain.pretrain_DNN(x=train_mnist, epochs=epochs_pretrain, batch_size=batch_size ,lr=learning_rate, show_progress=True)

### Backpropagation sur 2 DNN

Non pré-entrainé

In [ ]:
dnn_non_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
dnn_non_pretrain.retropropagation(X=train_mnist, Y=labels_train_mnist, epochs=epochs_backprop, lr=learning_rate, batch_size=batch_size, show_progress=True)

In [ ]:
dnn_non_pretrain.dbn.list_RBM[0].W.max()

In [ ]:
dnn_pretrain.dbn.list_RBM[0].W.max()

Pre-entrainé 

In [ ]:
dnn_pretrain.retropropagation(X=train_mnist, Y=labels_train_mnist, epochs=epochs_backprop, lr=learning_rate, batch_size=batch_size, show_progress=True)

### Test des deux DNN

In [ ]:
error_pretrain = dnn_pretrain.test_DNN(test_mnist, labels_test_mnist)

In [ ]:
error_non_pretrain = dnn_non_pretrain.test_DNN(test_mnist, labels_test_mnist)

In [ ]:
print(f"Error pretrain: {error_pretrain}")
print(f"Error non pretrain: {error_non_pretrain}")

### Test on concatenate both datasets

In [ ]:
x_train_test = torch.cat((train_mnist, test_mnist), 0)
y_train_test = torch.cat((labels_train_mnist, labels_test_mnist), 0)
error_pretrain_concat = dnn_pretrain.test_DNN(x_train_test, y_train_test)
error_non_pretrain_concat = dnn_non_pretrain.test_DNN(x_train_test, y_train_test)

In [ ]:
print(f"Error pretrain concat: {error_pretrain_concat}")
print(f"Error non pretrain concat: {error_non_pretrain_concat}")

In [ ]:
# save model to pkl
import pickle
with open("test_model", "wb") as file:
        pickle.dump(dnn_pretrain, file)

In [ ]:
with open("test_model", "rb") as file:
        model = pickle.load(file)

In [ ]:
torch.save(dnn_pretrain, "test_model.pt")

In [8]:
dnn_non_pretrain = DNN(layers_dbn, nb_classes=nb_classes)
dnn_non_pretrain.retropropagation_2(X=train_mnist, Y=labels_train_mnist, epochs=epochs_backprop, lr=learning_rate, batch_size=batch_size, show_progress=True)

Training DNN:   0%|          | 0/100 [00:00<?, ?epoch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)